In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import datetime
import requests as req
from pprint import pprint as pp
%matplotlib inline


In [2]:
df = pd.read_excel('Book Title List.xls',sheet_name="working",index_col = None,parse_dates=False)

*** formula/tFuncVar unknown FuncID:185


In [3]:
df.format.value_counts()

pdf                    420
Paperback              259
paperback                4
SmashWords               2
Kindle                   1
ePub                     1
Paperback - 2nd ed.      1
mobi                     1
Paperback - 1st ed.      1
SPANISH                  1
SW                       1
Name: format, dtype: int64

In [4]:
df['format'] = df['format'].str.lower()

In [5]:
list(df.format.value_counts().index)

['pdf',
 'paperback',
 'smashwords',
 'spanish',
 'kindle',
 'mobi',
 'sw',
 'paperback - 1st ed.',
 'paperback - 2nd ed.',
 'epub']

#### Create a list of dictionaries, each of which contains all of the data from a single title
First, loop over colA. If there's a value, create a dictionary. If not, continue.

Second, take the index of the colA value, add data to dictionary from each row, creating an array out of multiple column values.

Keep doing this until the loop finds another value.

In [6]:
titleList = []
def makeDict(df):
    for index, item in enumerate(df.title):
        #find the start and stop indicies for each title block
        if type(item) == str:
            index_start = index
            for subIndex, subItem in enumerate(df.title[index+1:]):
                if type(subItem) == str:
                    index_stop = subIndex+index+1
                    break
            titleList.append(df[index_start:index_stop].to_dict())
            #use the pandas to_dict method of slices of the dataframe
    return titleList
        

#### Flatten the email_usually column

In [7]:
titleDict = makeDict(df)

In [8]:
def moveFromEmailUsually(titleDict):
    for item in titleDict:
        for value in item['email_usually'].values():
            
            #move the emails found in this column
            if "@" in str(value):
                if 'email_from_email_usually' in item.keys():
                    
                    #gotta check for previous values and, if needed, make/append list
                    if type(item['email_from_email_usually'])==list:
                        item['email_from_email_usually'].append(value)
                    else:
                        values = item['email_from_email_usually']
                        values = [values]
                        values.append(value)
                        item['email_from_email_usually'] = values
                else:
                    item['email_from_email_usually'] = value
                
            # move the ISBNs found in this column. ACTUALLY, should do this when building all
            # ISBNs with their mappings. Commenting out for now.
            elif re.search(r"^\d+",str(value)):
                continue
#                 item['isbn_from_email_usually'] = value
            elif str(value)=='nan':
                continue
                
            #grab the rest of the trash in there
            else:
                item['extra_from_email_usually'] = value
    return titleDict
            

In [9]:
titleDict = moveFromEmailUsually(titleDict)

#### Flatten the people_usually column

In [10]:
def moveFromPeopleUsually(titleDict):
    for item in titleDict:
        for value in item['people_usually'].values():
            # Gonna just dump everything into an array in a single column. We can
            # flatten later if we think it's necessary.
            if str(value) == 'nan':
                continue 
            elif 'people_from_people_usually' in item.keys():
                if type(item['people_from_people_usually'])==list:
                    item['people_from_people_usually'].append(value)
                else:
                    values = item['people_from_people_usually']
                    values = [values]
                    values.append(value)
                    item['people_from_people_usually'] = values
            else:
                item['people_from_people_usually'] = value
    return titleDict

In [11]:
titleDict = moveFromPeopleUsually(titleDict)

#### Flatten the genre column

In [12]:
def moveFromGenres(titleDict):
    for item in titleDict:
        for value in item['genres'].values():
            # Gonna just dump everything into an array in a single column. We can
            # flatten later if we think it's necessary.
            if str(value) == 'nan':
                continue 
            elif 'genres_from_genres' in item.keys():
                if type(item['genres_from_genres'])==list:
                    item['genres_from_genres'].append(value)
                else:
                    values = item['genres_from_genres']
                    values = [values]
                    values.append(value)
                    item['genres_from_genres'] = values
            else:
                item['genres_from_genres'] = [value]
    return titleDict

In [13]:
titleDict = moveFromGenres(titleDict)

## I will come back to the isbn and format columns at the end since they are trickier and relate to one another in some cryptic way.

#### Flatten the bisac_code column
##### There are some weird things like "2nd ed" in the column. Handle that later.

In [14]:
def moveFromBisacCode(titleDict):
    for item in titleDict:
        for value in item['bisac_code'].values():
            # Gonna just dump everything into an array in a single column. We can
            # flatten later if we think it's necessary.
            if str(value) == 'nan':
                continue 
            elif 'bisac_code_flat' in item.keys():
                if type(item['bisac_code_flat'])==list:
                    item['bisac_code_flat'].append(value)
                else:
                    values = item['bisac_code_flat']
                    values = [values]
                    values.append(value)
                    item['bisac_code_flat'] = values
            else:
                item['bisac_code_flat'] = value
    return titleDict

In [15]:
titleDict = moveFromBisacCode(titleDict)

### Not yet, but here's how we're gonna do the ISBN flattening:

Okay, I think what's happening with the ISBNs is that the format column DOES map to the isbn_col (and the isbns in the email_usually col).

the first ISBN10 in the isbn_col is the pdf/ePub ISBN10. Below that is the ISBN13 for the ePub (it doesn't have a format corresponding to it, but i think that's what's happening here).

Then, the ISBN13 (usually 13) in the email_usually  column is the paperback's ISBN13. In the isbn_col, mapped to "paperback" is the paperback ISBN10, i think.

#### LOL, forgot to flatten the title and subtitle columns

In [16]:
def flattenTitles(titleDict):
    for item in titleDict:
        for value in item['title'].values():
            if str(value) == 'nan':
                continue 
            else:
                item['title_flat'] = value
        for subtitle in item['subtitle'].values():
            if str(subtitle) == 'nan':
                continue 
            else:
                item['subtitle_flat'] = subtitle
    return titleDict

In [17]:
titleDict = flattenTitles(titleDict)

In [41]:
titleDict[0]

{'email_from_email_usually': 'trish117@earthlink.net',
 'people_from_people_usually': 'Trish Iavarone',
 'genres_from_genres': ['Erotica'],
 'bisac_code_flat': 'FIC005000',
 'title_flat': '31 Affairs',
 'pdf_isbn10': '0744319498',
 'pdf_isbn13': '9780744319491',
 'paperback_isbn10': '0744321557',
 'paperback_isbn13': '9780744321555',
 'pdf_pub_date': datetime.datetime(2014, 9, 28, 0, 0),
 'pdf_copyright_date': datetime.datetime(2014, 6, 10, 0, 0),
 'paperback_pub_date': datetime.datetime(2014, 10, 13, 0, 0),
 'paperback_copyright_date': nan,
 'keywords_flat': 'affair, couples, fantasize, marriage, stimulate, steamy, forbidden, happy endings, sexual bucket list, erotica'}

#### Time to map the isbn, pub_date, and copyright_date to formats

In [19]:
#in pseudocode:
#loop over format column:
#    if key has value:
#        check email_usually, isbn_col, pub_date, and copyright_date fields, checking same key
#        generate new k:v pairs in the format formatvalue_field:field[key]
#            make sure you break up isbn_col by ISBN10 vs 13
#    if key has no value:
#        assign previous row's format formatting to that.
        

In [20]:


def mapFormat(titleDict):
    """This take the title list, in array-of-dicts form, and maps the ISBNs in
    the two columns to their corresponding format from the format column.
    It will log errors in parsing the ISBNs to a new field."""
    
    # will need this for the end to clean up
    format_values = list(df.format.value_counts().index)
    
    #Loop over the array of dictionaries
    for index, item in enumerate(titleDict):
        
        #then loop over the items in each title's format field
        for key, value in item['format'].items():
            if str(value) == 'nan':
                continue
                
            # handle PDF versions. Looks like PDF/ePub ISBN10 is first,
            # with ISBN13 in the next row.
            elif str(value) == 'pdf':
                try:
                    
                    # strip it of hyphens and trim any spaces, check for length
                    if len(item['isbn_col'][key].replace("-","").replace(" ",""))==10:
                        item['pdf_isbn10'] = item['isbn_col'][key].replace("-","").replace(" ","")
                        if len(item['isbn_col'][key+1].replace("-","").replace(" ",""))==13:
                            item['pdf_isbn13'] = item['isbn_col'][key+1].replace("-","").replace(" ","")
                        else:
                            print("pdf_ISBN13 at: ",index," is not 13 digits")
                            print("problem pdf_isbn13: ",item['isbn_col'][key+1].replace("-","").replace(" ",""))
                            item['pdf_isbn13'] = item['isbn_col'][key+1].replace("-","").replace(" ","")
                            item['error_parsing_pdf_isbn13'] = "not 13 digits"
                    else:
                        print("pdf_ISBN at: ",index," is not 10 digits")
                        item['error_parsing_pdf_isbn10'] = "not 10 digits"
                        item['pdf_isbn10'] = item['isbn_col'][key].replace("-","").replace(" ","")

                except Exception as e:
                    print(e)
                    item['error_parsing_pdf_isbn'] = e
                    
            # handle paperback versions. Typically ISBN13 is in the email_usually column
            # and then ISBN10 is in the same row, but in the isbn_col column.
            elif str(value) == 'paperback':
                try:
                    if len(item['isbn_col'][key].replace("-","").replace(" ",""))==10:
                        item['paperback_isbn10'] = item['isbn_col'][key].replace("-","").replace(" ","")
                        if len(item['email_usually'][key].replace("-","").replace(" ",""))==13:
                            item['paperback_isbn13'] = item['email_usually'][key].replace("-","").replace(" ","")
                        else:
                            print("paperback_ISBN13 at: ",index," is not 13 digits")
                            item['error_parsing_paperback_isbn13'] = "not 13 digits"
                            item['paperback_isbn13'] = item['email_usually'][key].replace("-","").replace(" ","")
                    else:
                        print("paperback_ISBN10 at: ",index," is not 10 digits")
                        item['paperback_isbn10'] = item['isbn_col'][key].replace("-","").replace(" ","")
                        item['error_parsing_paperback_isbn10'] = "not 10 digits"

                except Exception as e:
                    print(e)
                    item['error_parsing_paperback_isbn'] = e
                    
            #handle the special cases (Kindle, no value, )
            else:
                # LOL, this is definitely gonna fuck my algorithmic complexity, but yolo
                # Gotta check for the VERY FEW "weird" format values and remind us to manually fix
                for bookFormat in format_values:
                    if str(value) == bookFormat:
                        item['manual_check_special_ISBN_required'] = bookFormat
    return titleDict

In [21]:
titleDict = mapFormat(titleDict)

pdf_ISBN13 at:  2  is not 13 digits
problem pdf_isbn13:  978074307177
paperback_ISBN10 at:  30  is not 10 digits
'float' object has no attribute 'replace'
pdf_ISBN13 at:  62  is not 13 digits
problem pdf_isbn13:  978074419033
pdf_ISBN at:  124  is not 10 digits
paperback_ISBN10 at:  177  is not 10 digits
622
911
pdf_ISBN at:  343  is not 10 digits
pdf_ISBN at:  344  is not 10 digits
paperback_ISBN13 at:  349  is not 13 digits
pdf_ISBN at:  354  is not 10 digits
paperback_ISBN10 at:  433  is not 10 digits
paperback_ISBN13 at:  441  is not 13 digits


#### Map and flatten the pub dates to their respective formats

In [22]:
def mapDates(titleDict):
    """Gonna map the pub dates and copyright dates to their respective
    formats."""
    
    # will need this for the end to clean up
    format_values = list(df.format.value_counts().index)
    
    #Basically the same as the ISBN mapping logic
    for index, item in enumerate(titleDict):
        
        #then loop over the items in each title's format field
        for key, value in item['format'].items():
            if str(value) == 'nan':
                continue
                
            # handle PDF versions.
            elif str(value) == 'pdf':
                item['pdf_pub_date'] = item['pub_date'][key]
                item['pdf_copyright_date'] = item['copyright_date'][key]
            elif str(value) == 'paperback':
                item['paperback_pub_date'] = item['pub_date'][key]
                item['paperback_copyright_date'] = item['copyright_date'][key]
            else:
                for bookFormat in format_values:
                    if str(value) == bookFormat:
                        item[bookFormat+'_pub_date'] = item['pub_date'][key]
                        item[bookFormat+'_copyright_date'] = item['copyright_date'][key]
    return titleDict

    

In [23]:
titleDict = mapDates(titleDict)

In [24]:
titleDict[0]

[{'title': {0: '31 Affairs', 1: nan, 2: nan},
  'subtitle': {0: nan, 1: nan, 2: nan},
  'people_usually': {0: 'Trish Iavarone', 1: nan, 2: nan},
  'email_usually': {0: nan,
   1: 'trish117@earthlink.net',
   2: '978-0-7443-2155-5'},
  'genres': {0: 'Erotica', 1: nan, 2: nan},
  'isbn_col': {0: '0-7443-1949-8', 1: '978-0-7443-1949-1', 2: '0-7443-2155-7'},
  'format': {0: 'pdf', 1: nan, 2: 'paperback'},
  'bisac_code': {0: 'FIC005000', 1: nan, 2: nan},
  'pub_date': {0: datetime.datetime(2014, 9, 28, 0, 0),
   1: nan,
   2: datetime.datetime(2014, 10, 13, 0, 0)},
  'copyright_date': {0: datetime.datetime(2014, 6, 10, 0, 0), 1: nan, 2: nan},
  'contract_signed': {0: datetime.datetime(2019, 8, 26, 0, 0), 1: nan, 2: nan},
  'keywords': {0: 'affair, couples, fantasize, marriage, stimulate, steamy, forbidden, happy endings, sexual bucket list, erotica',
   1: nan,
   2: nan},
  'email_from_email_usually': 'trish117@earthlink.net',
  'people_from_people_usually': 'Trish Iavarone',
  'genres_fr

#### Time to put this bitch back into a flat DataFrame. Need to drop the old garbage columns.

In [25]:
# Check if contract_signed only ever has 1 value per title.
for item in titleDict:
    counter = 0
    for value in item['contract_signed']:
        if isinstance(item['contract_signed'][value],datetime.datetime):
            counter += 1
        if counter > 1:
            print(item['title_flat'])
        

Vinni's Bed
Vinni's Bed
Vinni's Bed


In [26]:
# Goddamnit, okay now I gotta add a second contract date to Vinni's stupid bed
for item in titleDict:
    try:
        if item['title_flat'] == "Vinni's Bed":
            item['SmashWords_contract_date'] = datetime.datetime(2019, 8, 12, 0, 0)
            item['pdf_contract_date'] = datetime.datetime(2014, 1, 1, 0, 0)
    except:
        print('no title_flat field')

no title_flat field


In [40]:
titleDict[0]

{'email_from_email_usually': 'trish117@earthlink.net',
 'people_from_people_usually': 'Trish Iavarone',
 'genres_from_genres': ['Erotica'],
 'bisac_code_flat': 'FIC005000',
 'title_flat': '31 Affairs',
 'pdf_isbn10': '0744319498',
 'pdf_isbn13': '9780744319491',
 'paperback_isbn10': '0744321557',
 'paperback_isbn13': '9780744321555',
 'pdf_pub_date': datetime.datetime(2014, 9, 28, 0, 0),
 'pdf_copyright_date': datetime.datetime(2014, 6, 10, 0, 0),
 'paperback_pub_date': datetime.datetime(2014, 10, 13, 0, 0),
 'paperback_copyright_date': nan,
 'keywords_flat': 'affair, couples, fantasize, marriage, stimulate, steamy, forbidden, happy endings, sexual bucket list, erotica'}

In [28]:
# flatten keywords
for item in titleDict:
    for key, value in item['keywords'].items():
        if str(value) == 'nan':
            continue
        else:
            item['keywords_flat'] = item['keywords'][key]


In [29]:
old_columns = ['title','subtitle','people_usually','email_usually',
               'genres','isbn_col','format','bisac_code','pub_date',
               'copyright_date','contract_signed','keywords']
for item in titleDict:
    for column in old_columns:
        item.pop(column)

In [30]:
df_flat = pd.DataFrame(titleDict)

In [31]:
# convert the isbns to strings so Excel doesn't scientific-notate them
# but probably won't work
df_flat['pdf_isbn10'] = df_flat['pdf_isbn10'].apply(str)
df_flat['pdf_isbn13'] = df_flat['pdf_isbn13'].apply(str)
df_flat['paperback_isbn10'] = df_flat['paperback_isbn10'].apply(str)
df_flat['paperback_isbn13'] = df_flat['paperback_isbn13'].apply(str)


In [32]:
# strip the times off the publishing dates that got added
df_flat['pdf_pub_date'] = [x.date() if isinstance(x,datetime.datetime) else x for x in df_flat['pdf_pub_date']]
df_flat['paperback_pub_date'] = [x.date() if isinstance(x,datetime.datetime) else x for x in df_flat['paperback_pub_date']]
df_flat['pdf_copyright_date'] = [x.date() if isinstance(x,datetime.datetime) else x for x in df_flat['pdf_copyright_date']]




In [33]:

for column in list(df_flat.columns):
    df_flat[column] = [', '.join(x) if type(x)==list else x for x in df_flat[column]]


In [34]:
df_flat['email_from_email_usually'][10:20]

10    rhondasedwards@gmail.com, stooke@dorchester2.K...
11                                 bryantarnold@cox.net
12                              rossrichdale@icloud.com
13                                 globuo01@hotmail.com
14                         rodmacdonald1@btinternet.com
15                                                  NaN
16                            justinleemcfar@yahoo.com 
17                              elenamanchita@yahoo.com
18                             lelandsmithson@gmail.com
19           Christieshary@aol.com, hwrites@verizon.net
Name: email_from_email_usually, dtype: object

In [35]:
# df_flat.to_csv('flattened_v1.csv')

In [36]:
list(df_flat.columns)

['email_from_email_usually',
 'people_from_people_usually',
 'genres_from_genres',
 'bisac_code_flat',
 'title_flat',
 'pdf_isbn10',
 'pdf_isbn13',
 'paperback_isbn10',
 'paperback_isbn13',
 'pdf_pub_date',
 'pdf_copyright_date',
 'paperback_pub_date',
 'paperback_copyright_date',
 'keywords_flat',
 'subtitle_flat',
 'error_parsing_pdf_isbn13',
 'extra_from_email_usually',
 'error_parsing_paperback_isbn10',
 'manual_check_special_ISBN_required',
 'mobi_pub_date',
 'mobi_copyright_date',
 'error_parsing_paperback_isbn',
 'spanish_pub_date',
 'spanish_copyright_date',
 'sw_pub_date',
 'sw_copyright_date',
 'error_parsing_pdf_isbn10',
 'error_parsing_pdf_isbn',
 'kindle_pub_date',
 'kindle_copyright_date',
 'error_parsing_paperback_isbn13',
 'epub_pub_date',
 'epub_copyright_date',
 'paperback - 1st ed._pub_date',
 'paperback - 1st ed._copyright_date',
 'paperback - 2nd ed._pub_date',
 'paperback - 2nd ed._copyright_date',
 'smashwords_pub_date',
 'smashwords_copyright_date',
 'SmashWords

In [37]:
#rearrange columns
df_flat_rearranged = df_flat[[
  'title_flat',
  'subtitle_flat',
 'people_from_people_usually',
 'email_from_email_usually',
 'genres_from_genres',
 'bisac_code_flat',
 'pdf_isbn10',
 'pdf_isbn13',
 'paperback_isbn10',
 'paperback_isbn13',
 'pdf_pub_date',
 'pdf_copyright_date',
 'paperback_pub_date',
 'paperback_copyright_date',
 'keywords_flat',
 'error_parsing_pdf_isbn13',
 'extra_from_email_usually',
 'error_parsing_paperback_isbn10',
 'manual_check_special_ISBN_required',
 'mobi_pub_date',
 'mobi_copyright_date',
 'error_parsing_paperback_isbn',
 'spanish_pub_date',
 'spanish_copyright_date',
 'sw_pub_date',
 'sw_copyright_date',
 'error_parsing_pdf_isbn10',
 'error_parsing_pdf_isbn',
 'kindle_pub_date',
 'kindle_copyright_date',
 'error_parsing_paperback_isbn13',
 'epub_pub_date',
 'epub_copyright_date',
 'paperback - 1st ed._pub_date',
 'paperback - 1st ed._copyright_date',
 'paperback - 2nd ed._pub_date',
 'paperback - 2nd ed._copyright_date',
 'smashwords_pub_date',
 'smashwords_copyright_date',
 'SmashWords_contract_date',
 'pdf_contract_date'
]]


In [38]:
df_flat_rearranged.rename(columns={
    'title_flat': 'Title',
    'subtitle_flat': 'Subtitle',
    'people_from_people_usually': 'People',
    'email_from_email_usually': 'Email',
    'genres_from_genres': 'Genres',
    'bisac_code_flat': 'BISAC Codes'},inplace=True)

In [39]:
df_flat_rearranged.to_csv('flattened_v2.csv')